In [1]:
import datetime as dt
from utility import (
    read_sql,
    fd_alive_funds,
    fd_basicinfo,
    fd_typeclass,
    fd_hshkiport,
    fd_assetportfolio,
    fd_qtfdnav,
    nearest_report_date
)
import pandas as pd
from PyFin.api import makeSchedule
from PyFin.api import BizDayConventions

In [2]:
# 获取回溯的报告日

today = dt.datetime.today()
date_3yrs_ago = today.replace(year=today.year - 3)
report_dates_begin = nearest_report_date(date_3yrs_ago.strftime("%Y%m%d"))
current_date = today.strftime("%Y%m%d")

In [7]:
# 获取所有目标债券型基金（清算结束日期晚于当前日）

security_ids = fd_alive_funds(report_dates_begin, 203)

In [8]:
# 获取相关组合情况

asset_port = fd_assetportfolio(security_ids, report_dates_begin, current_date).fillna(0)
asset_port["CONVBDRTO_TO_BDRTO"] = asset_port["CONVBDRTO"] / asset_port["BDRTO"] * 100

In [9]:
last_portfolio = asset_port.groupby("SECURITYID").last()[["BDRTO"]]
last_4_cov_portfolio = asset_port.groupby("SECURITYID").rolling(window=4).mean().groupby(level=0).last()[["CONVBDRTO_TO_BDRTO"]]
last_4_stk_portfolio = asset_port.groupby("SECURITYID").rolling(window=4).mean().groupby(level=0).last()[["EQUITYINVERTO"]]

In [10]:
agg_port = pd.concat([last_portfolio, last_4_cov_portfolio, last_4_stk_portfolio], axis=1).reset_index()
agg_port = pd.merge(agg_port, fd_typeclass(agg_port.SECURITYID.tolist(), current_date))
basic_info = fd_basicinfo(agg_port.SECURITYID.tolist())
agg_port = pd.merge(agg_port, basic_info)

In [11]:
agg_port.L3NAME.unique()

array(['普通债券型基金(一级A类)', '普通债券型基金(一级B/C类)', '普通债券型基金(二级A类)',
       '普通债券型基金(二级B/C类)', '中短期标准债券型基金', '偏债型基金', '普通债券型基金(可投转债A类)',
       '普通债券型基金(可投转债B类)', '指数债券型基金(A类)', '长期标准债券型基金(A类)', 'QDII债券基金',
       '可转换债券型基金(A类)', '可转换债券型基金(B/C类)', '指数债券型基金(B/C类)',
       '长期标准债券型基金(B/C类)', '封闭式普通债券型基金(可投转债)(B/C类)', '短期理财债券型基金(A类)',
       '短期理财债券型基金(B/C类)', '封闭式长期标准债券型基金(A类)', '封闭式长期标准债券型基金(B/C类)',
       '保本型基金', '封闭式普通债券型基金(可投转债)(A类)', '封闭式普通债券型基金(一级)(A类)',
       '债券型分级子基金(优先份额)', '债券型分级子基金(进取份额)', '封闭式普通债券型基金(二级)(A类)',
       '封闭式普通债券型基金(二级)(B/C类)', '封闭式普通债券型基金(二级)', '封闭式债券型分级子基金(优先份额)',
       '封闭式债券型分级子基金(进取份额)', '灵活配置型基金(股票上限95%)（A类）',
       '灵活配置型基金(股票上限95%)（B/C类）'], dtype=object)

In [12]:
# 分类
agg_port.loc[(agg_port["BDRTO"] >= 80) & (agg_port.L2NAME == "短期理财债券型基金"), "债券型（子类）"] = "短期纯债型基金"
agg_port.loc[(agg_port["BDRTO"] >= 80) & (agg_port["CONVBDRTO_TO_BDRTO"] >= 80), "债券型（子类）"] = "可转债基金"
agg_port.loc[(agg_port["BDRTO"] >= 80) & (agg_port["CONVBDRTO_TO_BDRTO"] >= 5) & (agg_port["CONVBDRTO_TO_BDRTO"] < 80) & (agg_port["EQUITYINVERTO"] <= 0.1), "债券型（子类）"] = "可投转债型债券基金"
agg_port.loc[(agg_port["BDRTO"] >= 80) & (agg_port["EQUITYINVERTO"] >= 0.1) & (~agg_port["债券型（子类）"].isin(["可投转债型债券基金", "可转债基金"])), "债券型（子类）"] = "可投股票型债券基金"
agg_port.loc[(agg_port["BDRTO"] >= 80) & (agg_port["L2NAME"] == "指数债券型基金") & (~agg_port["债券型（子类）"].isin(["可投转债型债券基金", "可转债基金"])), "债券型（子类）"] = "被动指数型债券基金"
agg_port.loc[pd.isnull(agg_port["债券型（子类）"]), "债券型（子类）"] = "中长期纯债型基金"

In [13]:
agg_port.groupby("债券型（子类）").count()

,SECURITYID,BDRTO,CONVBDRTO_TO_BDRTO,EQUITYINVERTO,L1CODE,L1NAME,L2CODE,L2NAME,L3CODE,L3NAME,FDNAME,SNAMECOMP,FSYMBOL,FDNATURE,INVESTSTYLE
债券型（子类）,,,,,,,,,,,,,,,
中长期纯债型基金,838,838,822,827,838,838,838,838,838,838,838,613,838,838,838
可投股票型债券基金,365,365,365,365,365,365,365,365,365,365,365,318,365,365,365
可投转债型债券基金,101,101,101,101,101,101,101,101,101,101,101,86,101,101,101
可转债基金,36,36,36,36,36,36,36,36,36,36,36,31,36,36,36
短期纯债型基金,47,47,47,47,47,47,47,47,47,47,47,46,47,47,47
被动指数型债券基金,13,13,13,13,13,13,13,13,13,13,13,10,13,13,13


## 1.1 Put it all totether

In [15]:
def create_bond_fund_info(trade_dt):
    trade_dt = dt.datetime.strptime(trade_dt, "%Y%m%d")
    date_3yrs_ago = trade_dt.replace(year=trade_dt.year - 3)
    report_dates_begin = nearest_report_date(date_3yrs_ago.strftime("%Y%m%d"))
    current_date = trade_dt.strftime("%Y%m%d")
    
    security_ids = fd_alive_funds(report_dates_begin, 203)
    basic_info = fd_basicinfo(security_ids, current_date)
    type_info = fd_typeclass(security_ids, current_date)
    asset_port = fd_assetportfolio(security_ids, report_dates_begin, current_date).fillna(0)
    asset_port["CONVBDRTO_TO_BDRTO"] = asset_port["CONVBDRTO"] / asset_port["BDRTO"] * 100
    last_portfolio = asset_port.groupby("SECURITYID").last()[["BDRTO"]]
    last_4_cov_portfolio = asset_port.groupby("SECURITYID").rolling(window=4).mean().groupby(level=0).last()[["CONVBDRTO_TO_BDRTO"]]
    last_4_stk_portfolio = asset_port.groupby("SECURITYID").rolling(window=4).mean().groupby(level=0).last()[["EQUITYINVERTO"]]

    df = pd.merge(basic_info, type_info, on="SECURITYID")
    df = pd.merge(df, last_portfolio, on="SECURITYID", how="left").drop_duplicates()
    df = pd.merge(df, last_4_stk_portfolio, on="SECURITYID", how="left").drop_duplicates()
    df = pd.merge(df, last_4_cov_portfolio, on="SECURITYID", how="left").drop_duplicates()
    
    df.loc[(df["BDRTO"] >= 80) & (df.L2NAME == "短期理财债券型基金"), "债券型（子类）"] = "短期纯债型基金"
    df.loc[(df["BDRTO"] >= 80) & (df["CONVBDRTO_TO_BDRTO"] >= 80), "债券型（子类）"] = "可转债基金"
    df.loc[(df["BDRTO"] >= 80) & (df["CONVBDRTO_TO_BDRTO"] >= 5) & (df["CONVBDRTO_TO_BDRTO"] < 80) & (df["EQUITYINVERTO"] <= 0.1), "债券型（子类）"] = "可投转债型债券基金"
    df.loc[(df["BDRTO"] >= 80) & (df["EQUITYINVERTO"] >= 0.1) & (~df["债券型（子类）"].isin(["可投转债型债券基金", "可转债基金"])), "债券型（子类）"] = "可投股票型债券基金"
    df.loc[(df["BDRTO"] >= 80) & (df["L2NAME"] == "指数债券型基金") & (~df["债券型（子类）"].isin(["可投转债型债券基金", "可转债基金"])), "债券型（子类）"] = "被动指数型债券基金"
    df.loc[pd.isnull(df["债券型（子类）"]), "债券型（子类）"] = "中长期纯债型基金"
    
    return df

In [16]:
%%time

df = create_bond_fund_info("20211111")#.groupby( "债券型（子类）")

Wall time: 10.7 s


In [17]:
%%time

create_bond_fund_info("20211111").groupby( "债券型（子类）").count()

Wall time: 18.1 s


,SECURITYID,FDNAME,SNAMECOMP,FSYMBOL,FDNATURE,INVESTSTYLE,L1CODE,L1NAME,L2CODE,L2NAME,L3CODE,L3NAME,BDRTO,EQUITYINVERTO,CONVBDRTO_TO_BDRTO
债券型（子类）,,,,,,,,,,,,,,,
中长期纯债型基金,872,872,644,872,872,872,872,872,872,872,872,872,839,828,823
可投股票型债券基金,365,365,318,365,365,365,365,365,365,365,365,365,365,365,365
可投转债型债券基金,101,101,86,101,101,101,101,101,101,101,101,101,101,101,101
可转债基金,36,36,31,36,36,36,36,36,36,36,36,36,36,36,36
短期纯债型基金,47,47,46,47,47,47,47,47,47,47,47,47,47,47,47
被动指数型债券基金,13,13,10,13,13,13,13,13,13,13,13,13,13,13,13


In [18]:
ddf = create_bond_fund_info("20211111")

In [19]:
ddf

,SECURITYID,FDNAME,SNAMECOMP,FSYMBOL,FDNATURE,INVESTSTYLE,L1CODE,L1NAME,L2CODE,L2NAME,L3CODE,L3NAME,BDRTO,EQUITYINVERTO,CONVBDRTO_TO_BDRTO,债券型（子类）
0,1030000011,华夏债券投资基金,华夏债券,001001,证券投资基金,收益型,3,债券基金,3.2,普通债券型基金,3.2.1,普通债券型基金(一级A类),89.97,2.5125,2.636154e+01,可投股票型债券基金
1,1030000012,华夏债券投资基金,华夏债券,001003,证券投资基金,收益型,3,债券基金,3.2,普通债券型基金,3.2.2,普通债券型基金(一级B/C类),113.82,2.0925,3.083827e+01,可投股票型债券基金
2,1030000014,华夏希望债券型证券投资基金,华夏希望债券,001011,证券投资基金,收益型,3,债券基金,3.2,普通债券型基金,3.2.3,普通债券型基金(二级A类),84.69,7.8500,9.399045e-01,可投股票型债券基金
3,1030000015,华夏希望债券型证券投资基金,华夏希望债券,001013,证券投资基金,收益型,3,债券基金,3.2,普通债券型基金,3.2.4,普通债券型基金(二级B/C类),94.13,6.4125,2.208407e+00,可投股票型债券基金
4,1030000022,国泰金龙债券证券投资基金,国泰金龙债券,020002,证券投资基金,收益型,3,债券基金,3.2,普通债券型基金,3.2.1,普通债券型基金(一级A类),130.35,1.0475,1.173590e+01,可投股票型债券基金
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429,1030002083,交银施罗德理财60天债券型证券投资基金,交银理财60天债券,519722,证券投资基金,收益型,3,债券基金,3.6,短期理财债券型基金,3.6.2,短期理财债券型基金(B/C类),92.11,0.0000,2.131628e-14,短期纯债型基金
1430,1030001814,易方达月月利理财债券型证券投资基金,易方达月月利理财债券,110050,证券投资基金,稳健成长型,3,债券基金,3.6,短期理财债券型基金,3.6.1,短期理财债券型基金(A类),42.43,0.0000,2.131628e-14,中长期纯债型基金
1431,1030001815,易方达月月利理财债券型证券投资基金,易方达月月利理财债券,110051,证券投资基金,稳健成长型,3,债券基金,3.6,短期理财债券型基金,3.6.2,短期理财债券型基金(B/C类),40.81,0.0000,2.131628e-14,中长期纯债型基金
1432,1030010003,易方达掌柜季季盈理财债券型证券投资基金,易方达掌柜季季盈理财债券,000833,证券投资基金,收益型,3,债券基金,3.6,短期理财债券型基金,3.6.1,短期理财债券型基金(A类),43.34,0.0000,1.421085e-14,中长期纯债型基金


# 2. 基金指数
--------------------

In [22]:
# 我们只计算指数的每日收益
# 在确定指数的基准日之后，可以直接使用收益计算指数的值。
# 使用 TQ_FD_DERIVEDN 获取基金的净值情况

start_dt = "2015-01-31"
final_trade_dt = "2021-11-09"
nav_type = "ACCUNITNAV"

rebalance_dates = [d.strftime("%Y%m%d") for d in makeSchedule(start_dt, final_trade_dt, tenor="3M", calendar="china.sse", dateRule=BizDayConventions.ModifiedFollowing)]

In [23]:
dfs = []

for i, trade_dt in enumerate(rebalance_dates[1:]):
    pre_trade_dt = rebalance_dates[i]
    print(pre_trade_dt, trade_dt)
    fund_info = create_bond_fund_info(trade_dt)
    pre_nav_info = fd_qtfdnav(fund_info.SECURITYID.tolist(), pre_trade_dt).rename(columns={nav_type: "PRE" + nav_type})
    nav_info = fd_qtfdnav(fund_info.SECURITYID.tolist(), trade_dt)

    total_df = pd.merge(fund_info, nav_info, on=["SECURITYID"])
    total_df = pd.merge(total_df, pre_nav_info, on=["SECURITYID"])
    total_df["chg."] = total_df[nav_type] / total_df["PRE" + nav_type] - 1.0
    res = total_df.groupby("债券型（子类）")[["chg."]].mean()
    dfs.append(res)

20150130 20150430
20150430 20150731
20150731 20151030
20151030 20160129
20160129 20160429
20160429 20160729
20160729 20161031
20161031 20170126
20170126 20170428
20170428 20170731
20170731 20171031
20171031 20180131
20180131 20180427
20180427 20180731
20180731 20181031
20181031 20190131
20190131 20190430
20190430 20190731
20190731 20191031
20191031 20200123
20200123 20200430
20200430 20200731
20200731 20201030
20201030 20210129
20210129 20210430
20210430 20210730
20210730 20211029
20211029 20211109


In [24]:
final_report = pd.concat(dfs, keys=rebalance_dates[1:]).reset_index()
final_report.pivot_table(index="level_0", columns="债券型（子类）", values="chg.").to_excel("030_债券型基金_bak.xlsx")

In [25]:
final_report.pivot_table(index="level_0", columns="债券型（子类）", values="chg.")

债券型（子类）,中长期纯债型基金,可投股票型债券基金,可投转债型债券基金,可转债基金,短期纯债型基金,被动指数型债券基金
level_0,,,,,,
20150430,0.034479,0.063696,0.032673,0.183849,NaN,0.014805
20150731,0.019400,-0.001548,0.009070,-0.181313,NaN,0.009040
20151030,0.020021,0.016197,0.016724,0.005966,NaN,0.022635
20160129,0.007229,-0.010521,0.008773,-0.119380,NaN,0.013658
20160429,0.001324,0.004405,0.001468,-0.009268,NaN,-0.000637
20160729,0.022094,0.016901,0.018205,-0.006460,NaN,0.020109
20161031,0.011922,0.014196,0.012508,0.032989,NaN,0.008745
20170126,-0.014330,-0.018991,-0.017139,-0.047551,NaN,-0.022332
20170428,-0.000164,-0.001997,-0.004151,-0.017770,NaN,-0.005754
